# Query Expansion

0) Just some imports

In [23]:
import re
import math
import numpy as np
import common as cm
from keras.preprocessing.text import Tokenizer
from numpy import dot 
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
tokenizer = Tokenizer()

# 1) Simple search engine

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [51]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeIDFs(self, documents):
        idfDict = {}
        for x in self.terms:
            sum = 0
            for y in documents:
                if x in y.raw_document:
                    sum += 1
            self.idfs.append(math.log(len(documents) / (1 + sum)))
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeCorM(self, documents):
        self.corM = []
        
        a = np.zeros((len(self.terms),len(documents)))
        for i in range(len(documents)):
            a[:,i] = np.array(documents[i].bow)
        
        n = []
        for i in a:
            i = i/math.sqrt(sum(i**2))
            n.append(i)
        
        t = np.transpose(n)
        self.corM = n@t
        for i in range(len(self.corM)):
            self.corM[i,i]=-1

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [52]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [53]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [86]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation

    ### TODO - complete this method; it should compute a BOW representation
    def computeBOW_Representation(self):
        self.bow = []
        for i in dictionary.terms:
            self.bow.append(self.tokens.count(i))
            
    ### TODO - complete this method; it should compute a TF representation
    def computeTF_Representation(self):
        self.tf = []
        for i in dictionary.terms:
            self.tf.append(self.tokens.count(i)/max(self.bow))
        
        
    
    ### TODO - complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    def computeTF_IDF_Representation(self):
        self.tf_idf = []
        for i in range(len(dictionary.terms)):
            self.tf_idf.append(self.tf[i]*self.dictionary.idfs[i])
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]
print(documents[0].tokens)

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.4) Compute IDFs here

In [87]:
### TODO COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print(res[:5])

[['www', 4.477336814478207], ['yahoo', 4.477336814478207], ['york', 4.477336814478207], ['young', 4.477336814478207], ['zdm', 4.477336814478207]]
[['and', 0.6706743247078867], ['the', 0.6931471805599453], ['for', 0.7161366987846441], ['learn', 1.0116009116784799], ['learning', 1.0433496099930604]]


1.5) Compute the document representations (for each document run computeRepresentations())

In [89]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print("[BoW] Bag of words: ")
print(documents[0].bow)
print("\n")
print("Tokens: ")
print(documents[0].tokens)
print("\n")
print("[TF] Term frequency: ")
print(documents[0].tf)
print("\n")
print("[TF-IDF] Term frequency-inverse document frequency: ")
print(documents[0].tf_idf)

[BoW] Bag of words: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 

1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [100]:
### TODO 
def getSimilarity(v1, v2):
    return dot(v1,v2)/(norm(v1)*norm(v2))
    #return sklearn.metrics.pairwise.cosine_similarity(v1,v2)

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [101]:
query = "machine learning"
#query = "academic research"
#query = "international conference"
#query = "international conference washington"

In [112]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.610806833803866 | DOC ID = 63
AI / Machine Learning Resources
 AI / Machine Learning Resources. General Machine Learning. The Journal
 of Machine Learning. MLnet Machine Learning Archive at GMD. The ... 


RANK = 2 WITH SIMILARITY = 0.5133103087314672 | DOC ID = 77
Machine Learning
 Machine Learning. Machine Learning Home Page (Editor) Machine Learning Home
 Page (Publisher) Machine Learning Online by Kluwer Academic Publishers: ... 


RANK = 3 WITH SIMILARITY = 0.5021511290837855 | DOC ID = 34
Machine Learning
 Machine Learning. Related Sites. Machine Learning Resources courtesy
 of David Aha A Machine Learning Tutorial a good overview of the ... 


RANK = 4 WITH SIMILARITY = 0.4599069618245262 | DOC ID = 28
Machine Learning
 6.858/18.428: Machine Learning. Available Lecture Notes. ... Defining models for
 machine learning. Learning conjunctions in the mistake-bounded model. ... 


RANK = 5 WITH SIMILARITY = 0.45627704064706714 | DOC ID = 20
Machine Learni

# 2) Query expansion

## 2.1) Correlation matrix

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [113]:
### TODO - COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
print(dictionary.corM)

[[-1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.         -1.          0.         ...  0.18898224  0.
   0.        ]
 [ 0.          0.         -1.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.18898224  0.         ... -1.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.         -1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -1.        ]]


2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [125]:
query = "machine"
#query = "algorithm"
#query = "learning"
#query = "conference"
#query = "research"
#query = "concept"

def suggestKeywords(query, dictionary):
    ind=dictionary.terms.index(query)
    val = dictionary.terms[np.argmax(dictionary.corM[ind])]
    key = [x for x in dictionary.corM[np.argmax(dictionary.corM[ind])]]
    print("Suggestion:\n " + str(val) + " -> " + str(max(key)))
        
suggestKeywords(query, dictionary)

Suggestion:
 learning -> 0.9700552531915929


# 2.2) Rocchio algorithm

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [141]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    
    sum_rel = np.array([documents[x].tf_idf for x in rel_docs]).sum(axis=0)
    sum_nrel = np.array([documents[x].tf_idf for x in nrel_docs]).sum(axis=0)

        
    qd.tf_idf = alpha*np.array(qd.tf_idf)+(beta*(1/len(rel_docs))*sum_rel)-(gamma*(1/len(nrel_docs))*sum_nrel)

    
    
    
    
    #####
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

RANK = 1 WITH SIMILARITY = 0.6884658756492513 | DOC ID = 36
Machine Learning Group
 [ Bristol CS | Index | Research | ML group | Student projects ] Machine
 Learning Research Group. Overview. The Machine Learning Research ... 
 Description: Research on higher-order concept learning, inductive logic programming, multi-agent learning systems,...


RANK = 2 WITH SIMILARITY = 0.6793147623055908 | DOC ID = 29
Machine Learning
 6.858/18.428: Machine Learning. ... This course deals with the following topics:
 Formal models of machine learning; Learning concepts from examples; ... 


RANK = 3 WITH SIMILARITY = 0.4382889847899004 | DOC ID = 13
UTCS Machine Learning Research Group
 Machine learning is the study of adaptive computational systems that
 improve their performance with experience. The UT Machine Learning ... 
 Description: Research on General Inductive Learning, Inductive Logic Programming, Natural Language Learning, Qualitati...


RANK = 4 WITH SIMILARITY = 0.4275352273369628 | DOC 

# 2.3) WordNet

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [142]:
from nltk.corpus import wordnet as wn

Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [143]:
wn.synsets('machine')

[Synset('machine.n.01'),
 Synset('machine.n.02'),
 Synset('machine.n.03'),
 Synset('machine.n.04'),
 Synset('machine.n.05'),
 Synset('car.n.01'),
 Synset('machine.v.01'),
 Synset('machine.v.02')]

2.3.3) Display all definitions (.definition()) for synsets (machine)

In [144]:
print([x.definition() for x in wn.synsets('machine')])

['any mechanical or electrical device that transmits or modifies energy to perform or assist in the performance of human tasks', 'an efficient person', 'an intricate organization that accomplishes its goals efficiently', 'a device for overcoming resistance at one point by applying force at some other point', 'a group that controls the activities of a political party', 'a motor vehicle with four wheels; usually propelled by an internal combustion engine', 'turn, shape, mold, or otherwise finish by machinery', 'make by machinery']


2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [145]:
print([x.hypernyms() for x in wn.synsets('machine')])

[[Synset('device.n.01')], [Synset('person.n.01')], [Synset('organization.n.01')], [Synset('mechanical_device.n.01')], [Synset('organization.n.01')], [Synset('motor_vehicle.n.01')], [Synset('shape.v.02')], [Synset('produce.v.02')]]


See: http://www.nltk.org/howto/wordnet.html
for more examples